# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6590c64dc0>
├── 'a' --> tensor([[ 1.3764, -0.6532,  1.7536],
│                   [-0.2487,  0.0902, -1.0240]])
└── 'x' --> <FastTreeValue 0x7f6590c64700>
    └── 'c' --> tensor([[ 0.4691, -2.1373, -0.6511,  1.9524],
                        [ 0.3680,  0.7250,  0.2813,  0.7259],
                        [-1.8187,  0.8027, -2.5309,  0.7617]])

In [4]:
t.a

tensor([[ 1.3764, -0.6532,  1.7536],
        [-0.2487,  0.0902, -1.0240]])

In [5]:
%timeit t.a

60.9 ns ± 0.0551 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6590c64dc0>
├── 'a' --> tensor([[-1.4551, -1.8430,  0.0342],
│                   [ 0.7127,  1.0481, -0.9404]])
└── 'x' --> <FastTreeValue 0x7f6590c64700>
    └── 'c' --> tensor([[ 0.4691, -2.1373, -0.6511,  1.9524],
                        [ 0.3680,  0.7250,  0.2813,  0.7259],
                        [-1.8187,  0.8027, -2.5309,  0.7617]])

In [7]:
%timeit t.a = new_value

67.8 ns ± 0.0665 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.3764, -0.6532,  1.7536],
               [-0.2487,  0.0902, -1.0240]]),
    x: Batch(
           c: tensor([[ 0.4691, -2.1373, -0.6511,  1.9524],
                      [ 0.3680,  0.7250,  0.2813,  0.7259],
                      [-1.8187,  0.8027, -2.5309,  0.7617]]),
       ),
)

In [10]:
b.a

tensor([[ 1.3764, -0.6532,  1.7536],
        [-0.2487,  0.0902, -1.0240]])

In [11]:
%timeit b.a

55.5 ns ± 0.0105 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3253, -0.1429, -0.8882],
               [ 1.1005, -0.5538,  0.7792]]),
    x: Batch(
           c: tensor([[ 0.4691, -2.1373, -0.6511,  1.9524],
                      [ 0.3680,  0.7250,  0.2813,  0.7259],
                      [-1.8187,  0.8027, -2.5309,  0.7617]]),
       ),
)

In [13]:
%timeit b.a = new_value

483 ns ± 0.0785 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

886 ns ± 11.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 8.81 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 330 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 274 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f6592d51fd0>
├── 'a' --> tensor([[[ 1.3764, -0.6532,  1.7536],
│                    [-0.2487,  0.0902, -1.0240]],
│           
│                   [[ 1.3764, -0.6532,  1.7536],
│                    [-0.2487,  0.0902, -1.0240]],
│           
│                   [[ 1.3764, -0.6532,  1.7536],
│                    [-0.2487,  0.0902, -1.0240]],
│           
│                   [[ 1.3764, -0.6532,  1.7536],
│                    [-0.2487,  0.0902, -1.0240]],
│           
│                   [[ 1.3764, -0.6532,  1.7536],
│                    [-0.2487,  0.0902, -1.0240]],
│           
│                   [[ 1.3764, -0.6532,  1.7536],
│                    [-0.2487,  0.0902, -1.0240]],
│           
│                   [[ 1.3764, -0.6532,  1.7536],
│                    [-0.2487,  0.0902, -1.0240]],
│           
│                   [[ 1.3764, -0.6532,  1.7536],
│                    [-0.2487,  0.0902, -1.0240]]])
└── 'x' --> <FastTreeValue 0x7f6590c64790>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.1 µs ± 45.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f64ba4edfa0>
├── 'a' --> tensor([[ 1.3764, -0.6532,  1.7536],
│                   [-0.2487,  0.0902, -1.0240],
│                   [ 1.3764, -0.6532,  1.7536],
│                   [-0.2487,  0.0902, -1.0240],
│                   [ 1.3764, -0.6532,  1.7536],
│                   [-0.2487,  0.0902, -1.0240],
│                   [ 1.3764, -0.6532,  1.7536],
│                   [-0.2487,  0.0902, -1.0240],
│                   [ 1.3764, -0.6532,  1.7536],
│                   [-0.2487,  0.0902, -1.0240],
│                   [ 1.3764, -0.6532,  1.7536],
│                   [-0.2487,  0.0902, -1.0240],
│                   [ 1.3764, -0.6532,  1.7536],
│                   [-0.2487,  0.0902, -1.0240],
│                   [ 1.3764, -0.6532,  1.7536],
│                   [-0.2487,  0.0902, -1.0240]])
└── 'x' --> <FastTreeValue 0x7f64ba4d6be0>
    └── 'c' --> tensor([[ 0.4691, -2.1373, -0.6511,  1.9524],
                        [ 0.3680,  0.7250,  0.2813,  0.7259],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 68 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.1 µs ± 146 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.3764, -0.6532,  1.7536],
                [-0.2487,  0.0902, -1.0240]],
       
               [[ 1.3764, -0.6532,  1.7536],
                [-0.2487,  0.0902, -1.0240]],
       
               [[ 1.3764, -0.6532,  1.7536],
                [-0.2487,  0.0902, -1.0240]],
       
               [[ 1.3764, -0.6532,  1.7536],
                [-0.2487,  0.0902, -1.0240]],
       
               [[ 1.3764, -0.6532,  1.7536],
                [-0.2487,  0.0902, -1.0240]],
       
               [[ 1.3764, -0.6532,  1.7536],
                [-0.2487,  0.0902, -1.0240]],
       
               [[ 1.3764, -0.6532,  1.7536],
                [-0.2487,  0.0902, -1.0240]],
       
               [[ 1.3764, -0.6532,  1.7536],
                [-0.2487,  0.0902, -1.0240]]]),
    x: Batch(
           c: tensor([[[ 0.4691, -2.1373, -0.6511,  1.9524],
                       [ 0.3680,  0.7250,  0.2813,  0.7259],
                       [-1.8187,  0.8027, -2.5309,  0.7617]],
         

In [26]:
%timeit Batch.stack(batches)

78.8 µs ± 229 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.3764, -0.6532,  1.7536],
               [-0.2487,  0.0902, -1.0240],
               [ 1.3764, -0.6532,  1.7536],
               [-0.2487,  0.0902, -1.0240],
               [ 1.3764, -0.6532,  1.7536],
               [-0.2487,  0.0902, -1.0240],
               [ 1.3764, -0.6532,  1.7536],
               [-0.2487,  0.0902, -1.0240],
               [ 1.3764, -0.6532,  1.7536],
               [-0.2487,  0.0902, -1.0240],
               [ 1.3764, -0.6532,  1.7536],
               [-0.2487,  0.0902, -1.0240],
               [ 1.3764, -0.6532,  1.7536],
               [-0.2487,  0.0902, -1.0240],
               [ 1.3764, -0.6532,  1.7536],
               [-0.2487,  0.0902, -1.0240]]),
    x: Batch(
           c: tensor([[ 0.4691, -2.1373, -0.6511,  1.9524],
                      [ 0.3680,  0.7250,  0.2813,  0.7259],
                      [-1.8187,  0.8027, -2.5309,  0.7617],
                      [ 0.4691, -2.1373, -0.6511,  1.9524],
                      [ 0.3680,  

In [28]:
%timeit Batch.cat(batches)

148 µs ± 500 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

316 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
